# How to use this notebook

**Before running this notebook:**
<br>
Set up the following file structure (namely, the `raw/`, `processed/`, and `archive/` folders within `data/`). Populate `raw/` with the data that we have been given. 

```
data/
├── raw/
│   ├── district.json
│   ├── 2024-09-11_District_A_Benchmarks.json
│   ├── 2024-09-11_District_A_Scores.json
│   └── ... (future raw data files)
├── processed/
└── archive/
README.md
```

**When you run this notebook, it will...**
- Rename the raw files to follow a consistent format
- Pull out each individual table 
- Save each table as a `.csv` in `/data/processed`

All names will be formatted as follows: `[district]_[table]_[date].[extension]`
- e.g. `a_all_2024-09-11.json`
- e.g. `a_scores_2024-09-11.csv`

**After running this notebook:**
<br>
You can import tables indiviudally by using `pd.read_csv()`.


In [2]:
import os
import json
import pandas as pd

In [3]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_path = os.path.join(repo_root, 'predicting-proficiency', 'data')

## Renaming files

In [4]:
file_names = [
    ('2024-09-11 District A Benchmarks.json', 'a_benchmarks_2024-09-11.json'),
    ('2024-09-11 District A Scores.json', 'a_scores_2024-09-11.json'),
    ('2024-09-11 District A Vendor Student Usage.json', 'a_vendorUsage_2024-09-11.json'),
    ('district.json', 'a_all_2024-08-29.json'),
    ('vendorProducts_202409111049.csv', 'a_vendorKey_2024-09-11.csv')
]

for old_name, new_name in file_names:
    old_path = os.path.join(data_path, 'raw', old_name)
    new_path = os.path.join(data_path, 'raw', new_name)
    if os.path.exists(old_path) and not os.path.exists(new_path):
        os.rename(old_path, new_path)

In [5]:
with open('data/raw/a_all_2024-08-29.json', 'r') as file:
    data = json.loads(file.read())

with open('data/raw/a_scores_2024-09-11.json', 'r') as file:
    scores_json = json.loads(file.read())

with open('data/raw/a_benchmarks_2024-09-11.json', 'r') as file:
    benchmarks_json = json.loads(file.read())

with open('data/raw/a_vendorUsage_2024-09-11.json', 'r') as file:
    vendorUsage_json = json.loads(file.read())

## Saving .csvs

In [6]:
scores_data = scores_json[list(scores_json.keys())[0]]
scores_df = pd.DataFrame(scores_data)

benchmarks_df = pd.DataFrame(benchmarks_json['benchmarks'])
schools_df = pd.DataFrame(data['schools'])
courseSections_df = pd.DataFrame(data['courseSections'])
courseSectionRosters_df = pd.DataFrame(data['courseSectionRosters'])

vendorUsage_data = vendorUsage_json[list(vendorUsage_json.keys())[0]]
vendorUsage_df = pd.DataFrame(vendorUsage_data)

vendorKey_df = pd.read_csv('data/raw/a_vendorKey_2024-09-11.csv')

In [7]:
data_path = os.path.join(repo_root, 'predicting-proficiency', 'data')

processed_dir = os.path.join(data_path, 'processed')
if os.path.exists(processed_dir):
    for file in os.listdir(processed_dir):
        file_path = os.path.join(processed_dir, file)
        if os.path.isfile(file_path):
            os.unlink(file_path)

os.makedirs(os.path.dirname(data_path), exist_ok=True)
benchmarks_df.to_csv(os.path.join(data_path, 'processed', 'a_benchmarks_2024-09-11.csv'), index=False)
schools_df.to_csv(os.path.join(data_path, 'processed', 'a_schools_2024-08-29.csv'), index=False)
courseSections_df.to_csv(os.path.join(data_path, 'processed', 'a_courseSections_2024-08-29.csv'), index=False)
courseSectionRosters_df.to_csv(os.path.join(data_path, 'processed', 'a_courseSectionRosters_2024-08-29.csv'), index=False)
scores_df.to_csv(os.path.join(data_path, 'processed', 'a_scores_2024-09-11.csv'), index=False)
vendorUsage_df.to_csv(os.path.join(data_path, 'processed', 'a_vendorUsage_2024-09-11.csv'), index=False)

You can stop here! The rest of the notebook is just flagging issues in the data that need to be resolved.

## Potential issues

We currently have two DataFrames with info on vendors: `vendorUsage_df` which contains usage stats with numerical ids for each vendor, and `vendorKey_df` which is a lookup table containing the name and id of each vendor. We will merge the two and export it as a single csv.

Actually we won't yet because the ids don't match up.

In [8]:
# vendorUsage_df = vendorUsage_df.merge(
#     vendorKey_df[['vendorId', 'productName']], 
#     on='vendorId', 
#     how='left'
# )

In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [12]:
set1 = set(vendorUsage_df['vendorId'].unique())
set2 = set(vendorKey_df['vendorId'].unique())

missing_keys = set1 - set2
num_missing = len(missing_keys)

print(f"Number of vendorIds in vendorUsage with no key: {num_missing}")
print(f"VendorIds with no key: {missing_keys}")

Number of vendorIds in vendorUsage with no key: 21
VendorIds with no key: {63616, 22410, 37269, 37017, 39195, 72609, 36130, 27172, 62508, 62509, 62382, 25405, 63431, 63432, 52447, 52448, 61544, 61673, 7273, 7035, 23165}


Scores

Vendor

In [29]:
print(vendorUsage_df.shape)
print(vendorUsage_df.columns)

(158007, 7)
Index(['id', 'vendorId', 'studentId', 'active', 'year', 'usageTypeId',
       'weeklyUsageMinutes'],
      dtype='object')
